In [38]:
import numpy as np
import random
from abc import abstractmethod

In [6]:
# generate random hamd
hand = random.sample(list(range(136)), 13)
hand_tensor = np.zeros(37, dtype=np.float32)
for tile in hand:
    hand_tensor[tile // 4] += 1.0

assert hand_tensor.sum() == 13.0

In [55]:
hand_tensor = np.array([
    0, 2, 2, 1, 2, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
])

In [42]:
class ShantenSolver:
    @abstractmethod
    def _init(self, hand_tensor):
        assert hand_tensor.sum() <= 14.0
        self.hand_tensor = hand_tensor
    
    @abstractmethod
    def solve(self):
        pass

In [40]:
class NormalHandShantenSolver(ShantenSolver):
    def _init(self, hand_tensor):
        super()._init(hand_tensor)
        self.n_melds = (14 - hand_tensor.sum()) // 3
        self.n_mentsu = self.n_melds + sum(hand_tensor[27:] >= 3.0)
        self.n_tatsu = 0
        self.n_toitsu = sum(hand_tensor[27:] == 2.0)
        self.n_kuttsuki = 0
        self.best_shanten = 14
        self.tune = 0

    def add_shuntsu(self, idx):
        self.hand_tensor[idx:idx + 3] -= 1
        self.n_mentsu += 1
    
    def remove_shuntsu(self, idx):
        self.hand_tensor[idx:idx + 3] += 1
        self.n_mentsu -= 1
    
    def add_koutsu(self, idx):
        self.hand_tensor[idx] -= 3
        self.n_mentsu += 1
    
    def remove_koutsu(self, idx):
        self.hand_tensor[idx] += 3
        self.n_mentsu -= 1

    def add_toitsu(self, idx):
        self.hand_tensor[idx] -= 2
        self.n_toitsu += 1
    
    def remove_toitsu(self, idx):
        self.hand_tensor[idx] += 2
        self.n_toitsu -= 1

    def add_taatsu(self, idx, which):
        self.hand_tensor[idx:idx+3] -= 1
        self.hand_tensor[idx+which] += 1
        self.n_tatsu += 1
    
    def remove_taatsu(self, idx, which):
        self.hand_tensor[idx:idx+3] += 1
        self.hand_tensor[idx+which] -= 1
        self.n_tatsu -= 1

    def add_kuttsuki(self, idx, tune):
        self.hand_tensor[idx] -= 1
        self.n_kuttsuki += 1
        if tune:
            self.tune += 1
    
    def remove_kuttsuki(self, idx, tune):
        self.hand_tensor[idx] += 1
        self.n_kuttsuki -= 1
        if tune:
            self.tune -= 1

    def solve(self, idx):
        while idx < 27:
            if self.hand_tensor[idx]:
                break
            idx += 1
        
        if idx == 27:
            shanten = min(self.best_shanten, 8 - self.n_mentsu * 2 - self.n_tatsu - self.n_toitsu)
            potential_mentsu = max(0, self.n_toitsu - 1) + self.n_tatsu
            if potential_mentsu + self.n_mentsu > 4:
                shanten += potential_mentsu + self.n_mentsu - 4
            if self.tune and not self.n_toitsu:
                shanten += 1
            self.best_shanten = min(self.best_shanten, shanten)
            return

        num = idx % 9

        def process_shuntsu(tune=False):
            if num < 7 and self.hand_tensor[idx+2]:
                if self.hand_tensor[idx+1]:
                    self.add_shuntsu(idx)
                    self.solve(idx)
                    self.remove_shuntsu(idx)
                self.add_taatsu(idx, 1)
                self.solve(idx)
                self.remove_taatsu(idx, 1)
            
            if num < 8 and self.hand_tensor[idx+1]:
                self.add_taatsu(idx, 2)
                self.solve(idx)
                self.remove_taatsu(idx, 2)

            self.add_kuttsuki(idx, tune)
            self.solve(idx)
            self.remove_kuttsuki(idx, tune)

        if self.hand_tensor[idx] == 4:
            self.add_koutsu(idx)
            process_shuntsu(tune=True)
            self.solve(idx+1)
            self.remove_koutsu(idx)

            self.add_toitsu(idx)
            process_shuntsu()
            self.remove_toitsu(idx)

        elif self.hand_tensor[idx] == 3:
            self.add_koutsu(idx)
            self.solve(idx+1)
            self.remove_koutsu(idx)

            self.add_toitsu(idx)
            self.solve(idx)
            self.remove_toitsu(idx)
        
        elif self.hand_tensor[idx] == 2:
            self.add_toitsu(idx)
            self.solve(idx+1)
            self.remove_toitsu(idx)

            process_shuntsu()
        
        elif self.hand_tensor[idx] == 1:
            process_shuntsu()


In [41]:
solver = NormalHandShantenSolver()
solver._init(hand_tensor)
solver.solve(0)
print(solver.best_shanten)

0


In [43]:
class ChitoitsuHandShantenSolver(ShantenSolver):
    def _init(self, hand_tensor):
        super()._init(hand_tensor)
    
    def solve(self):
        return 6 - sum(hand_tensor >= 2.0)

In [44]:
solver = NormalHandShantenSolver()
solver._init(hand_tensor)
solver.solve(0)
print(solver.best_shanten)

0


In [58]:
class KokushiHandShantenSolver(ShantenSolver):
    def _init(self, hand_tensor):
        super()._init(hand_tensor)
    
    def solve(self):
        yaochu_tensor = np.zeros(37)
        yaochu_tensor[[0, 8, 9, 17, 18, 26, 27, 28, 29, 30, 31, 32, 33, 34]] = 1.0
        yaochu_arr = self.hand_tensor * yaochu_tensor
        return 13 - sum(yaochu_arr >= 1.0) - (max(yaochu_arr) - 1.0)

In [59]:
solver = KokushiHandShantenSolver()
solver._init(hand_tensor)
print(solver.solve())

11.0
